In [1]:
from collections import Counter, defaultdict
import math
import pandas as pd

In [2]:
def build_ngrams(tokens, n):
    #Return n-gram counts as Counter
    ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
    return Counter(ngrams)

In [3]:
def good_turing_smoothing(ngram_counts, vocab_size, n):
    #Returns smoothed probabilities for given n-gram counts
    total_seen = sum(ngram_counts.values())   # N
    freq_of_freq = Counter(ngram_counts.values())  # Nc

    N1 = freq_of_freq.get(1, 0)   # number of ngrams that occurred once
    
    # Total possible ngrams
    total_possible = (vocab_size ** n)
    num_unseen = total_possible - len(ngram_counts)

    # Probability mass reserved for unseen ngrams
    p_unseen = N1 / total_seen if total_seen > 0 else 0
    p_each_unseen = p_unseen / num_unseen if num_unseen > 0 else 0

    smoothed_probs = {}

    for ng, c in ngram_counts.items():
        Nc = freq_of_freq[c]
        Nc1 = freq_of_freq.get(c+1, 0)

        if Nc1 > 0:
            c_star = (c+1) * (Nc1 / Nc)
        else:
            c_star = c  # fallback

        smoothed_probs[ng] = c_star / total_seen

    return smoothed_probs, p_each_unseen

In [ ]:
def sentence_probability(sentence_tokens, model_probs, p_unseen, n):
    
    ngrams = [tuple(sentence_tokens[i:i+n]) for i in range(len(sentence_tokens)-n+1)]
    log_prob = 0.0

    for ng in ngrams:
        if ng in model_probs:
            log_prob += math.log(model_probs[ng])
        else:
            log_prob += math.log(p_unseen) if p_unseen > 0 else float("-inf")

    return log_prob  # log probability

In [5]:
train_df = pd.read_csv("train.csv")
val_df   = pd.read_csv("validation.csv")
test_df  = pd.read_csv("test.csv")

In [7]:
def extract_sentences(df, column="sentence"):
    """Return list of sentences from a plain text column."""
    return df[column].dropna().tolist()


In [8]:
train_sentences = extract_sentences(train_df)
val_sentences   = extract_sentences(val_df)
test_sentences  = extract_sentences(test_df)

print(f"Training set: {len(train_sentences)} sentences")
print(f"Validation set: {len(val_sentences)} sentences")
print(f"Test set: {len(test_sentences)} sentences")


Training set: 1238 sentences
Validation set: 154 sentences
Test set: 156 sentences


In [9]:
train_tokens = [word for sent in train_sentences for word in sent.split()]
vocab = set(train_tokens)
V = len(vocab)

In [10]:
models = {}
for n in range(1, 5):
    counts = build_ngrams(train_tokens, n)
    smoothed_probs, p_unseen = good_turing_smoothing(counts, V, n)
    models[n] = (smoothed_probs, p_unseen)

In [11]:
def perplexity(sentences, model_probs, p_unseen, n):
    #lower perplexity -> better model
    total_log_prob = 0.0
    total_tokens = 0

    for sent in sentences:
        tokens = sent.split()
        lp = sentence_probability(tokens, model_probs, p_unseen, n)
        total_log_prob += lp
        total_tokens += len(tokens)

    if total_tokens == 0:
        return float("inf")

    return math.exp(-total_log_prob / total_tokens)

In [12]:
for split_name, split_sentences in [("Validation", val_sentences), ("Test", test_sentences)]:
    print(f"\n{split_name} Perplexities:")
    for n in range(1, 5):
        model_probs, p_unseen = models[n]
        ppl = perplexity(split_sentences, model_probs, p_unseen, n)
        print(f"{n}-gram model: Perplexity = {ppl:.4f}")


Validation Perplexities:
1-gram model: Perplexity = inf
2-gram model: Perplexity = 5336695.0689
3-gram model: Perplexity = 7465180223.2015
4-gram model: Perplexity = 2245490105677.9380

Test Perplexities:
1-gram model: Perplexity = inf
2-gram model: Perplexity = 6018942.6891
3-gram model: Perplexity = 7895610246.4466
4-gram model: Perplexity = 2293854547559.6338
